这里做语言流向的桑基图，首先，为了和文本分析中的舆论趋势对应，我们这里首先做2020年数据中实际到预期的桑基图。

我们将一个程序员`当前实际在用的技术`到`预期使用的技术`绘制为整个技术流向网络中的一条有向边，如果这条边存在，则为其增加边权，统计每个节点的入度（被预期的那些）和出度之和，就能反映出舆论趋势的变化。

In [2]:
# 导入数据
import pandas as pd

data1=pd.read_csv("../data/2020/survey_results_public.csv")
languages = data1[["LanguageWorkedWith", "LanguageDesireNextYear"]]
languages = languages.dropna()

In [3]:
# 对网络中的边进行统计
langs_transition = dict()
for row in languages.itertuples():
    for from_lang in row.LanguageWorkedWith.split(';'):
        for to_lang in row.LanguageDesireNextYear.split(';'):
            # if from_lang == to_lang:
            #     continue
            langs_transition[(from_lang, to_lang)] = langs_transition.get((from_lang, to_lang), 0) + 1

In [4]:
# 准备桑基图节点
links = []

from_langs = set()
to_langs = set()

for key, value in langs_transition.items():
    if value < 5000:
        continue
    from_langs.add(key[0])
    to_langs.add(key[1])
    links.append({'source': 'from ' + key[0], 'target':'to ' + key[1], 'value': value})

In [5]:
# 接下来尝试对语言节点进行颜色的手工标注
all_langs = set()
for row in languages.itertuples():
    all_langs.update(set(row.LanguageWorkedWith.split(';')))
    all_langs.update(set(row.LanguageDesireNextYear.split(';')))
print(sorted(list(all_langs)))

['Assembly', 'Bash/Shell/PowerShell', 'C', 'C#', 'C++', 'Dart', 'Go', 'HTML/CSS', 'Haskell', 'Java', 'JavaScript', 'Julia', 'Kotlin', 'Objective-C', 'PHP', 'Perl', 'Python', 'R', 'Ruby', 'Rust', 'SQL', 'Scala', 'Swift', 'TypeScript', 'VBA']


In [6]:
# 参考https://github.com/ozh/github-colors/blob/master/colors.json中给出的语言-颜色对照
palette = {
    'Assembly': '#6E4C13', 'Bash/Shell/PowerShell': '#012456', 'C': '#555555', 'C#': '#178600',
    'C++': '#f34b7d', 'Dart': '#00B4AB', 'Go': '#00ADD8', 'HTML/CSS': '#e34c26', 'Haskell': '#5e5086',
    'Java': '#b07219', 'JavaScript': '#f1e05a', 'Julia': '#a270ba', 'Kotlin': '#28430A',
    'Objective-C': '#438eff', 'PHP': '#4F5D95', 'Perl': '#0298c3', 'Python': '#3572A5',
    'R': '#198CE7', 'Ruby': '#701516', 'Rust': '#dea584', 'SQL': '#e38c00', 'Scala': '#c22d40',
    'Swift': '#F05138', 'TypeScript': '#2b7489', 'VBA': '#867db1'
}


In [9]:
# 准备节点
nodes = []
for lang in from_langs:
    nodes.append({'name': 'from ' + lang, 'itemStyle':{'color': palette[lang]}})
for lang in to_langs:
    nodes.append({'name': 'to ' + lang, 'itemStyle':{'color': palette[lang]}})

In [10]:
# 绘制桑基图
from pyecharts import options as opts
from pyecharts.charts import Sankey

Sankey(
    init_opts=opts.InitOpts(
        width='1000px',
        height='800px',
        bg_color='#fff'
    )
).add(
    '',
    nodes,
    links,
    node_gap=0,
    node_width=160,
    pos_right='5%',
    node_align='justify',
    focus_node_adjacency=True,
    linestyle_opt=opts.LineStyleOpts(curve=0.5, opacity=0.2, color="source"),
    label_opts=opts.LabelOpts(position='inside', color='white'),
    itemstyle_opts=opts.ItemStyleOpts(border_color="#fff"),
).render('output/language_sankey.html')

'f:\\reimagined-octo-guide\\network_analysis\\output\\language_sankey.html'

在这个分析中，入度大于出度的那些节点对应的语言有着更高的舆论趋势。从上面的结果中我们可以看出，有较为明显趋势的语言如下：
1. 扩大：Python（1/4希望留用），TypeScript，还有本年度不在热门列的Rust、Go和Kotlin。
2. 缩小：Java（1/6希望留用）转向Python、Javascript、Go等，C（几乎消亡）

有趣的是，上述七种语言，除TypeScript和Kotlin之外的其余五种有一些共性：
1. 在文本分析部分中，词频都高于JavaScript
2. 都是典型的后端语言。

我们的猜想是，这可能是前端程序员有着“TypeScript是下一代Javascript”的普遍认知，因而没有产生较多的讨论，而后端技术的更迭往往伴随着大量的讨论。这或许在一定程度上说明了前端界技术认知的一致性，以及我们所设想的“后端技术壁垒”，即后端技术的更迭往往伴随着大量程序员的广泛讨论。

Kotlin是Android端平台上替代Java的下一代开发语言，严格来讲不能归于前后端中的一方。但Kotlin的例子同样说明，关于技术的认知是否一致，与社区中讨论的热度有关联。

当然我们必须说明的是，这个部分基于可视化而作，略显感性，是对我们结论的一个辅证。